In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math
%load_ext tensorboard

2022-12-31 03:02:51.890116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 03:02:52.401230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-31 03:02:52.401297: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-31 03:02:54.336181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
input_file_name = 'dataset_week1.json'
filepath = os.path.join(os.getenv('NFL_DATA'), input_file_name)
df = pd.read_json(filepath)
df.head()

,nfl_id,opponent_coords,teammate_coords,ball_coords,game_id,play_id,frame_id,coords,pff_position_lined_up,quarter,down,yards_to_go,absolute_yardline_number,personnel_o,defenders_in_box,offense_formation,score_delta
0,41263,"[[-2.61, 0.45], [-1.29, 0.15], [0.1, -0.54], [...","[[-1.99, -2.12], [1.26, -1.79], [2.75, -2.14],...","[0, 0]",2021090900,97,6,"[-5.03, -1.74]",LEO,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
1,42403,"[[3, 0.18], [1.73, -0.37], [0.1, -0.54], [-1.2...","[[2.75, -2.14], [1.26, -1.79], [-1.99, -2.12],...","[0, 0]",2021090900,97,6,"[8.71, -2.34]",ROLB,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
2,44955,"[[1.73, -0.37], [0.1, -0.54], [3, 0.18], [-1.2...","[[2.75, -2.14], [-1.99, -2.12], [-5.03, -1.74]...","[0, 0]",2021090900,97,6,"[1.26, -1.79]",DRT,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
3,53441,"[[-1.29, 0.15], [0.1, -0.54], [-2.61, 0.45], [...","[[-5.03, -1.74], [1.26, -1.79], [2.75, -2.14],...","[0, 0]",2021090900,97,6,"[-1.99, -2.12]",LILB,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0
4,53504,"[[1.73, -0.37], [3, 0.18], [0.1, -0.54], [-1.2...","[[1.26, -1.79], [-1.99, -2.12], [8.71, -2.34],...","[0, 0]",2021090900,97,6,"[2.75, -2.14]",RE,1,3,2,43,"1 RB, 1 TE, 3 WR",6.0,SHOTGUN,0


In [3]:
num_games = df["game_id"].nunique()
num_plays = df["play_id"].nunique()
num_players = df["nfl_id"].nunique()

grouped_df = df.groupby(["nfl_id","game_id","play_id"])
num_playerPlays = grouped_df.ngroups
min_frames = min(grouped_df.count()["frame_id"])
max_frames = max(grouped_df.count()["frame_id"])
print(f"The data contains {num_playerPlays} unique 'Rushes' from {num_players} players over {num_plays} plays (across {num_games} games). \nThe shortest play has {min_frames} frames and the longest play has {max_frames} frames")

The data contains 4764 unique 'Rushes' from 430 players over 979 plays (across 16 games). 
The shortest play has 10 frames and the longest play has 99 frames


In [19]:
def split_sequence(sequence, max_frames):
    y = np.array(sequence.pop(-1))
    X = np.zeros((max_frames - 1,2))
    seq_len = len(sequence)
    for i in range(seq_len):
        X[(max_frames - 1 - seq_len) + i] = sequence[i]
    return X, y

In [20]:
def make_features(df, max_frames):
    X, y  = list(), list()
    grouped_df = df.groupby(["nfl_id","game_id","play_id"])
    for group_name, group_df in grouped_df:
        coords = group_df["coords"].tolist()
        X1, y1 = split_sequence(coords, max_frames)
        X.append(X1)
        y.append(y1)

    return np.stack(X), np.stack(y)

In [21]:
X, y = make_features(df, max_frames)

split = 0.8
num_train = round(split * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]

In [26]:
X.shape

(4764, 98, 2)

In [27]:
y.shape

(4764, 2)

The dataset is as follows:

`X` is a 3D tensor with shape (`num_rushes`, `max_frame - 1`, 2). In other words, it is a stack of 2D "rush" tensors with shape (`max_frame - 1`, 2), where `max_frame` is the max number of frames across all rushes in the data (e.g. 99 frames or 9.9 seconds). The last dimension has a value of 2 because of the x and y coordinates we are tracking. We can add more features in that dimension.

The Y matrix is a 2D tensor with shape (`num_rushes`, 2)

In [30]:
# set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# create the model
model = tf.keras.Sequential()
# model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten(), input_shape=(96, 2, 3)))
model.add(tf.keras.layers.LSTM(32, input_shape = X.shape[1:]))
model.add(tf.keras.layers.Dense(2))

# compile and fit the model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, verbose=0)

In [31]:
err = 0
for i,j in zip(model.predict(X_test), y_test):
    err += math.dist(i,j)
print(err/len(y_test)) # avg prediction error (yards)

30/30 [==============================] - 1s 26ms/step
0.03094482434815262
